<div style = "background-color:indigo"><center>
<h1 style="font-size: 50px; font-weight: bold; color:goldenrod; border-top: 3px solid goldenrod; padding-top: 10px">OC SWITRS GIS Data Processing</h1>
<div style="font-size: 35px; font-weight: bold; color: goldenrod"> Part 1 - Feature Class Geoprocessing</div>
<div style="font-size: 30px; font-weight: bold; color: goldenrod; border-bottom: 3px solid goldenrod; padding-bottom: 20px">v.2.1, May 2025</div>
</center></div>

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">1. Preliminaries</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.1 Referencing Libraries and Initialization</h2>

If needed to reset the kernel, please run the following cell:

In [ ]:
#%reset

Instantiating python libraries for the project

In [1]:
# Import Python libraries
import os, json, pytz, math, arcpy, arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from arcpy import metadata as md

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.2. Project and Workspace Variables</h2>

Define and maintain project, workspace, ArcGIS, and data-related variables

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project and Geodatabase Paths</h3>

Define the ArcGIS pro project variables

In [2]:
# Current notebook directory
notebookDir = os.getcwd()

# Define the project folder (parent directory of the current working directory)
projectFolder = os.path.dirname(os.getcwd())

Which running environment this notebook is using? (1 = Visual Studio Code, 2 = ArcGIS Pro)

In [3]:
runEnv = input("Enter the running environment (1=VSCode, 2=ArcGIS Pro): ")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">ArcGIS Pro Paths</h3>

ArcGIS pro related paths

In [ ]:
# OCSWITRS project AGP path
agpFolder = os.path.join(projectFolder, "AGPSWITRS")

# AGP APRX file name and path
aprxName = "AGPSWITRS.aprx"
aprxPath = os.path.join(agpFolder, aprxName)

# ArcGIS Pro project geodatabase and path
gdbName = "AGPSWITRS.gdb"
gdbPath = os.path.join(agpFolder, gdbName)

aprx = arcpy.mp.ArcGISProject(aprxPath)
# ArcGIS pro project
if runEnv == "1":  # VSCode
    print("Running in VSCode (project = aprxPath)")
    aprx = arcpy.mp.ArcGISProject(aprxPath)
elif runEnv == "2":  # ArcGIS Pro
    print("Running in ArcGIS Pro (project = CURRENT)")
    aprx = arcpy.mp.ArcGISProject('CURRENT')

# Close all map views
aprx.closeViews()

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace
# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True
# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

Running in VSCode (project = aprxPath)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Folder Paths</h3>

In [5]:
# Raw data folder path
rawDataFolder = os.path.join(projectFolder, "data", "raw")

# Maps folder path
mapsFolder = os.path.join(projectFolder, "maps")

# Layers folder path
layersFolder = os.path.join(projectFolder, "layers")
layersTemplate = os.path.join(layersFolder, "templates")

# Layouts folder path
layoutsFolder = os.path.join(projectFolder, "layouts")

# Notebooks folder path
notebooksFolder = os.path.join(projectFolder, "notebooks")
codebookPath = os.path.join(projectFolder, "scripts", "codebook", "cb.json")

Geodatabase feature datasets paths (directories)

In [6]:
# RawData feature dataset in the geodatabase
gdbRawData = os.path.join(gdbPath, "raw")

# RawData feature dataset in the geodatabase
gdbSupportingData = os.path.join(gdbPath, "supporting")

# AnalysisData feature dataset in the geodatabase
gdbAnalysisData = os.path.join(gdbPath, "analysis")

# HotSpotData feature dataset in the geodatabase
gdbHotspotData = os.path.join(gdbPath, "hotspots")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Data Folder Paths</h3>

The most current raw data files cover the periods from 01/01/2013 to 09/30/2024. The data files are already processed in the R scripts and imported into the project's geodatabase.

In [7]:
# Add the start date of the raw data to a new python datetime object
dateStart = datetime(2012, 1, 1)

# Add the end date of the raw data to a new python datetime object
dateEnd = datetime(2024, 12, 31)

# Define time and date variables
timeZone = pytz.timezone("US/Pacific")
today = datetime.now(timeZone)
dateUpdated = today.strftime("%B %d, %Y")
timeUpdated = today.strftime("%I:%M %p")

Define date strings for metadata

In [8]:
# String defining the years of the raw data
mdYears = f"{dateStart.year}-{dateEnd.year}"

# String defining the start and end dates of the raw data
mdDates = f"Data from {dateStart.strftime('%B %d, %Y')} to {dateEnd.strftime('%B %d, %Y')}"

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Codebook</h3>

In [9]:
# Load the JSON file from directory and store it in a variable
with open(codebookPath) as jsonFile:
    codebook = json.load(jsonFile)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">JSON CIM Exports</h3>

Creating a function to export the CIM JSON files to disk.

In [10]:
def export_cim(cimType, cimObject, cimName):
    """Export a CIM object to a file in both native (MAPX, PAGX, LYRX) and JSON CIM formats."""
    match cimType:
        # When the CIM object is a map
        case "map":
            # Export the CIM object to a MAPX file
            print(f"Exporting {cimName} map to MAPX...")
            cimObject.exportToMAPX(os.path.join(mapsFolder, cimName+".mapx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} map to JSON...\n")
            with open(os.path.join(mapsFolder, cimName+".mapx"), "r") as f:
                data = f.read()
            with open(os.path.join(mapsFolder, cimName+".json"), "w") as f:
                f.write(data)
        
        # When the CIM object is a layout
        case "layout":
            # Export the CIM object to a PAGX file
            print(f"Exporting {cimName} layout to PAGX...")
            cimObject.exportToPAGX(os.path.join(layoutsFolder, cimName+".pagx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} layout to JSON...\n")
            with open(os.path.join(layoutsFolder, cimName+".pagx"), "r") as f:
                data = f.read()
            with open(os.path.join(layoutsFolder, cimName+".json"), "w") as f:
                f.write(data)
        
        # When the CIM object is a layer
        case "layer":
            # Export the CIM object to a LYRX file
            print(f"Exporting {cimName} layer to LYRX...")
            # Reformat the name of the output file
            for m in aprx.listMaps():
                for l in m.listLayers():
                    if l == cimObject:
                        cimNewName = m.name.title()+"Map-"+l.name.replace("OCSWITRS ", "")
            # Save the layer to a LYRX file
            arcpy.management.SaveToLayerFile(cimObject, os.path.join(layersFolder, cimNewName + ".lyrx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} layer to JSON...\n")
            with open(os.path.join(layersFolder, cimNewName+".lyrx"), "r") as f:
                data = f.read()
            with open(os.path.join(layersFolder, cimNewName+".json"), "w") as f:
                f.write(data)

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.3. ArcGIS Pro Workspace</h2>

Set the workspace and environment settings for the ArcGIS Pro project

In [11]:
# Set the workspace and environment to the root of the project geodatabase
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True

# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.4. Map and Layout Lists</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Maps</h3>

In [12]:
# List of maps to be created for the project
mapList = ["collisions", "crashes", "parties", "victims", "injuries", "fatalities", "fhs100m1km", "fhs150m2km", "fhs100m5km", "fhsRoads500ft", "ohsRoads500ft", "roadCrashes",
           "roadHotspots", "roadBuffers", "roadSegments", "roads", "pointFhs", "pointOhs", "popDens", "houDens", "areaCities", "areaBlocks", "summaries", "analysis", "regression"]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Layouts</h3>

In [13]:
# List or layouts to be created for the project
layoutList = ["maps", "injuries", "hotspots", "roads", "points", "densities", "areas"]

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.5. Clean Up Data</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete Feature Classes</h3>

Clean up the feature classes in the geodatabase for the Analysis and Hotspot Feature Datasets

In [14]:
for d in ["analysis", "hotspots"]:
    print(f"Dataset: {d}")
    for f in arcpy.ListFeatureClasses(feature_dataset = d):
        print(f"- Removing {f} feature class from the project...")
        arcpy.management.Delete(f)

Dataset: analysis
Dataset: hotspots


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete Maps</h3>

Clean up the maps in the project structure

In [15]:
for m in aprx.listMaps():
    print(f"- Removing {m.name} map from the project...")
    aprx.deleteItem(m)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete Layouts</h3>

Clean up the layouts in the project structure

In [16]:
for l in aprx.listLayouts():
    print(f"- Removing {l.name} layout from the project...")
    aprx.deleteItem(l)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [17]:
# Save the project
aprx.save()

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">2. Geodatabase Operations</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.1. Raw Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Paths</h3>

Paths to raw data geodatabase feature classes

In [18]:
# Paths to raw data feature classes
victims = os.path.join(gdbRawData, "victims")
parties = os.path.join(gdbRawData, "parties")
crashes = os.path.join(gdbRawData, "crashes")
collisions = os.path.join(gdbRawData, "collisions")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Fields</h3>

Obtain a list of fields for each raw data geodatabase feature class

In [19]:
# Fields for the raw data feature classes
victimsFields = [f.name for f in arcpy.ListFields(victims)] # victims field list
partiesFields = [f.name for f in arcpy.ListFields(parties)] # parties field list
crashesFields = [f.name for f in arcpy.ListFields(crashes)] # crashes field list
collisionsFields = [f.name for f in arcpy.ListFields(collisions)] # collisions field list

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Row Counts</h3>

Count rows in each of the raw data geodatabase feature classes

In [20]:
# Get the count for the raw data feature classes
victimsCount = int(arcpy.management.GetCount(victims)[0])
partiesCount = int(arcpy.management.GetCount(parties)[0])
crashesCount = int(arcpy.management.GetCount(crashes)[0])
collisionsCount = int(arcpy.management.GetCount(collisions)[0])

print(f"\nRaw Data Counts:\n- Victims: {victimsCount:,}\n- Parties: {partiesCount:,}\n- Crashes: {crashesCount:,}\n- Collisions: {collisionsCount:,}")


Raw Data Counts:
- Victims: 296,827
- Parties: 367,407
- Crashes: 170,195
- Collisions: 437,258


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Collisions Feature Class Aliases</h3>

Adding feature class alias for the collisions feature class

In [21]:
# Collisions feature class alias
collisionsAlias = "OCSWITRS Collisions"

# Collisions feature class
arcpy.AlterAliasName(collisions, collisionsAlias)
print(f"Collisions: {arcpy.GetMessages()}")

Collisions: Start Time: Wednesday, May 7, 2025 12:11:13 PM
Row Count = 437258
Succeeded at Wednesday, May 7, 2025 12:11:13 PM (Elapsed Time: 0.06 seconds)


Adding field aliases to the collisions feature class

In [22]:
# Collisions field aliases
for f in collisionsFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = collisions,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

	Match 1: caseId (Case ID)
	Match 2: cid (Crash ID)
	Match 3: pid (Party ID)
	Match 4: vid (Victim ID)
	Match 5: crashTag (Crash Tag)
	Match 6: partyTag (Party Tag)
	Match 7: victimTag (Victim Tag)
	Match 8: partyNumber (Party Number)
	Match 9: victimNumber (Victim Number)
	Match 10: crashesCaseTag (Crashes Tag)
	Match 11: partiesCaseTag (Parties Tag)
	Match 12: victimsCaseTag (Victims Tag)
	Match 16: combinedInd (Combined Indicator)
	Match 17: crashesCidCount (Total Collisions in Crashes Dataset)
	Match 18: partiesCidCount (Total Collisions in Parties Dataset)
	Match 19: victimsCidCount (Total Collisions in Victims Dataset)
	Match 20: partiesPidCount (Total Parties in Parties Dataset)
	Match 21: victimsPidCount (Total Parties in Victims Dataset)
	Match 22: victimsVidCount (Total Victims in Victims Dataset)
	Match 23: city (City)
	Match 24: placeType (Place Type)
	Match 25: dateDatetime (Crash Date and Time)
	Match 26: dateYear (Crash Year)
	Match 27: dateQuarter (Crash Quarter)
	Match

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes Feature Class Aliases</h3>

Adding feature class alias for the crashes feature class

In [23]:
# Crashes feature class alias
crashesAlias = "OCSWITRS Crashes"

# Crashes feature class
arcpy.AlterAliasName(crashes, crashesAlias)
print(f"Crashes: {arcpy.GetMessages()}")

Crashes: Start Time: Wednesday, May 7, 2025 12:13:16 PM
Succeeded at Wednesday, May 7, 2025 12:13:17 PM (Elapsed Time: 0.31 seconds)


Adding field aliases to the crashes feature class

In [24]:
# Crashes field aliases
for f in crashesFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = crashes,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

	Match 1: caseId (Case ID)
	Match 2: cid (Crash ID)
	Match 5: crashTag (Crash Tag)
	Match 10: crashesCaseTag (Crashes Tag)
	Match 17: crashesCidCount (Total Collisions in Crashes Dataset)
	Match 23: city (City)
	Match 25: dateDatetime (Crash Date and Time)
	Match 26: dateYear (Crash Year)
	Match 27: dateQuarter (Crash Quarter)
	Match 28: dateMonth (Crash Month)
	Match 29: dateWeek (Crash Week)
	Match 30: dateDay (Crash Day)
	Match 31: dateProcess (Process Date)
	Match 32: dtYear (Crash Year)
	Match 33: dtQuarter (Crash Quarter)
	Match 34: dtMonth (Crash Month)
	Match 35: dtYearWeek (Crash Year Week)
	Match 36: dtWeekDay (Crash Weekday)
	Match 37: dtMonthDay (Crash Month Day)
	Match 38: dtYearDay (Crash Year Day)
	Match 39: dtHour (Crash Hour)
	Match 40: dtMinute (Crash Minute)
	Match 41: dtDst (Crash Daylight Saving Time)
	Match 42: dtZone (Crash Time Zone)
	Match 43: collDate (Collision Date Reported)
	Match 44: collTime (Collision Time Reported)
	Match 45: accidentYear (Crash Year)
	

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Parties Feature Class Aliases</h3>

Adding feature class alias for the parties feature class

In [25]:
# Parties feature class alias
partiesAlias = "OCSWITRS Parties"

# Parties feature class
arcpy.AlterAliasName(parties, partiesAlias)
print(f"Parties: {arcpy.GetMessages()}")

Parties: Start Time: Wednesday, May 7, 2025 12:14:37 PM
Succeeded at Wednesday, May 7, 2025 12:14:38 PM (Elapsed Time: 0.29 seconds)


Adding field aliases to the parties feature class

In [26]:
# Parties field aliases
for f in partiesFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = parties,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

	Match 1: caseId (Case ID)
	Match 2: cid (Crash ID)
	Match 3: pid (Party ID)
	Match 6: partyTag (Party Tag)
	Match 8: partyNumber (Party Number)
	Match 11: partiesCaseTag (Parties Tag)
	Match 18: partiesCidCount (Total Collisions in Parties Dataset)
	Match 20: partiesPidCount (Total Parties in Parties Dataset)
	Match 25: dateDatetime (Crash Date and Time)
	Match 26: dateYear (Crash Year)
	Match 27: dateQuarter (Crash Quarter)
	Match 28: dateMonth (Crash Month)
	Match 29: dateWeek (Crash Week)
	Match 30: dateDay (Crash Day)
	Match 31: dateProcess (Process Date)
	Match 32: dtYear (Crash Year)
	Match 33: dtQuarter (Crash Quarter)
	Match 34: dtMonth (Crash Month)
	Match 35: dtYearWeek (Crash Year Week)
	Match 36: dtWeekDay (Crash Weekday)
	Match 37: dtMonthDay (Crash Month Day)
	Match 38: dtYearDay (Crash Year Day)
	Match 39: dtHour (Crash Hour)
	Match 40: dtMinute (Crash Minute)
	Match 41: dtDst (Crash Daylight Saving Time)
	Match 42: dtZone (Crash Time Zone)
	Match 43: collDate (Collisio

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Victims Feature Class Aliases</h3>

Adding feature class alias for the victims feature class

In [27]:
# Victims feature class alias
victimsAlias = "OCSWITRS Victims"

# Victims feature class
arcpy.AlterAliasName(victims, victimsAlias)
print(f"Victims: {arcpy.GetMessages()}")

Victims: Start Time: Wednesday, May 7, 2025 12:17:49 PM
Succeeded at Wednesday, May 7, 2025 12:17:50 PM (Elapsed Time: 0.28 seconds)


Adding field aliases to the victims feature class

In [28]:
# Victims field aliases
for f in victimsFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = victims,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

	Match 1: caseId (Case ID)
	Match 2: cid (Crash ID)
	Match 3: pid (Party ID)
	Match 4: vid (Victim ID)
	Match 7: victimTag (Victim Tag)
	Match 8: partyNumber (Party Number)
	Match 9: victimNumber (Victim Number)
	Match 12: victimsCaseTag (Victims Tag)
	Match 19: victimsCidCount (Total Collisions in Victims Dataset)
	Match 21: victimsPidCount (Total Parties in Victims Dataset)
	Match 22: victimsVidCount (Total Victims in Victims Dataset)
	Match 25: dateDatetime (Crash Date and Time)
	Match 26: dateYear (Crash Year)
	Match 27: dateQuarter (Crash Quarter)
	Match 28: dateMonth (Crash Month)
	Match 29: dateWeek (Crash Week)
	Match 30: dateDay (Crash Day)
	Match 31: dateProcess (Process Date)
	Match 32: dtYear (Crash Year)
	Match 33: dtQuarter (Crash Quarter)
	Match 34: dtMonth (Crash Month)
	Match 35: dtYearWeek (Crash Year Week)
	Match 36: dtWeekDay (Crash Weekday)
	Match 37: dtMonthDay (Crash Month Day)
	Match 38: dtYearDay (Crash Year Day)
	Match 39: dtHour (Crash Hour)
	Match 40: dtMinu

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.2. Supporting Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Paths</h3>

Paths to the supporting data geodatabase feature classes

In [29]:
# Paths to supporting data feature classes
boundaries = os.path.join(gdbSupportingData, "boundaries")
cities = os.path.join(gdbSupportingData, "cities")
blocks = os.path.join(gdbSupportingData, "blocks")
roads = os.path.join(gdbSupportingData, "roads")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Fields</h3>

Obtain the list fields of the supporting data geodatabase feature classes

In [30]:
# Fields for the supporting data feature classes
boundariesFields = [f.name for f in arcpy.ListFields(boundaries)] # boundaries field list
citiesFields = [f.name for f in arcpy.ListFields(cities)] # cities field list
blocksFields = [f.name for f in arcpy.ListFields(blocks)] # censusBlocks field list
roadsFields = [f.name for f in arcpy.ListFields(roads)] # roads field list

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Row Counts</h3>

Count rows in each of the supporting data geodatabase feature classes

In [31]:
# Get the count for the supporting data feature classes
boundariesCount = int(arcpy.management.GetCount(boundaries)[0])
citiesCount = int(arcpy.management.GetCount(cities)[0])
blocksCount = int(arcpy.management.GetCount(blocks)[0])
roadsCount = int(arcpy.management.GetCount(roads)[0])

# Print the counts
print(f"Supporting Data Counts:\n- Boundaries: {boundariesCount:,}\n- Cities: {citiesCount:,}\n- Census Blocks: {blocksCount:,}\n- Roads: {roadsCount:,}")

Supporting Data Counts:
- Boundaries: 1
- Cities: 46
- Census Blocks: 26,864
- Roads: 59,144


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Roads Feature Class</h3>

Adding feature class alias for the roads feature class

In [32]:
# Roads feature class alias
roadsAlias = "OCSWITRS Roads"

# Roads feature class
arcpy.AlterAliasName(roads, roadsAlias)
print(f"Roads: {arcpy.GetMessages()}")

Roads: Start Time: Wednesday, May 7, 2025 12:19:39 PM
Row Count = 59144
Succeeded at Wednesday, May 7, 2025 12:19:39 PM (Elapsed Time: 0.08 seconds)


Adding field aliases to the roads feature class

In [33]:
# Roads field aliases
for f in roadsFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = roads,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

	Match 178: placeName (Road City)
	Match 176: roadName (Road Name)
	Match 177: roadId (Road ID)
	Match 179: roadCat (Road Category)
	Match 180: roadLength (Road Length (in miles))
	Match 181: roadsPrimary (Number of Primary Road Segments)
	Match 182: roadsSecondary (Number of Secondary Road Segments)
	Match 183: roadsLocal (Number of Local Road Segments)
	Match 184: roadLengthMean (Mean Road Length)
	Match 185: roadLengthSum (Total Road Length)
Start Time: Wednesday, May 7, 2025 12:19:53 PM
Succeeded at Wednesday, May 7, 2025 12:19:53 PM (Elapsed Time: 0.26 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Census Blocks Feature Class</h3>

Adding feature class alias for the census blocks feature class

In [34]:
# Census Blocks feature class alias
blocksAlias = "OCSWITRS Census Blocks"

# Census Blocks feature class
arcpy.AlterAliasName(blocks, blocksAlias)
print(f"USC 2020 Census Blocks: {arcpy.GetMessages()}")

USC 2020 Census Blocks: Start Time: Wednesday, May 7, 2025 12:19:53 PM
Succeeded at Wednesday, May 7, 2025 12:19:53 PM (Elapsed Time: 0.26 seconds)


Adding field aliases to the census blocks feature class

In [35]:
# Census Blocks field aliases
for f in blocksFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = blocks,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

	Match 186: populationTotal (Total Population)
	Match 187: housingUnitsTotal (Total Housing Units)
	Match 188: householdsTotal (Total Households)
	Match 189: housingUnitsVacant (Vacant Housing Units)
	Match 190: housingUnitsMortgage (Housing Units with Mortgage)
	Match 191: housingUnitsOwned (Housing Units Owned)
	Match 192: housingUnitsRenterOccupied (Renter Occupied Housing Units)
	Match 193: vacancyRate (Vacancy Rate)
	Match 194: housingUnitsOwnerOccupied (Owner Occupied Housing Units)
	Match 195: homeownershipRate (Homeownership Rate)
	Match 196: populationDensity (Population Density)
	Match 197: housingDensity (Housing Density)
Start Time: Wednesday, May 7, 2025 12:20:15 PM
Succeeded at Wednesday, May 7, 2025 12:20:16 PM (Elapsed Time: 0.36 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Cities Feature Class</h3>

Adding feature class alias for the cities feature class

In [36]:
# Cities feature class alias
citiesAlias = "OCSWITRS Cities"

# Cities feature class
arcpy.AlterAliasName(cities, citiesAlias)
print(f"Cities: {arcpy.GetMessages()}")

Cities: Start Time: Wednesday, May 7, 2025 12:20:15 PM
Succeeded at Wednesday, May 7, 2025 12:20:16 PM (Elapsed Time: 0.36 seconds)


Adding field aliases to the cities feature class

In [37]:
# Cities field aliases
for f in citiesFields:
    if f in list(codebook.keys()):
        print(f"\tMatch {codebook[f]['varOrder']}: {f} ({codebook[f]['label']})")
        arcpy.management.AlterField(
            in_table = cities,
            field = f,
            new_field_alias = codebook[f]['label']
        )
print(arcpy.GetMessages())

	Match 23: city (City)
	Match 24: placeType (Place Type)
	Match 164: cityAreaSqmi (City Area)
	Match 167: cityPopTotal (City Total Population)
	Match 168: cityHouTotal (City Total Housing Units)
	Match 165: cityPopDens (City Population Density)
	Match 166: cityHouDens (City Housing Density)
	Match 169: cityPopAsian (City Asian Population)
	Match 170: cityPopBlack (City Black Population)
	Match 171: cityPopHispanic (City Hispanic Population)
	Match 172: cityPopWhite (City White Population)
	Match 173: cityVehicles (Number of Commuting Vehicles)
	Match 174: cityTravelTime (Aggregate Travel Time to Work)
	Match 175: cityMeanTravelTime (Mean Travel Time to work Per Vehicle)
Start Time: Wednesday, May 7, 2025 12:20:33 PM
Succeeded at Wednesday, May 7, 2025 12:20:33 PM (Elapsed Time: 0.40 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Boundaries Feature Class</h3>

Adding feature class alias for the boundaries feature class

In [38]:
# Boundaries feature class alias
boundariesAlias = "OCSWITRS Boundaries"

# Boundaries feature class
arcpy.AlterAliasName(boundaries, boundariesAlias)
print(f"Boundaries: {arcpy.GetMessages()}")

Boundaries: Start Time: Wednesday, May 7, 2025 12:20:33 PM
Succeeded at Wednesday, May 7, 2025 12:20:33 PM (Elapsed Time: 0.40 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [39]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.3. Data Enrichment Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Paths</h3>

In [40]:
collisions1 = os.path.join(gdbRawData, "collisions1")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Feature Class Joins</h3>

In [41]:
# Join the collisions feature class with the censusBlocks feature class
arcpy.analysis.SpatialJoin(
    target_features = collisions,
    join_features = blocks,
    out_feature_class = collisions1,
    join_operation = "JOIN_ONE_TO_ONE",
    join_type = "KEEP_ALL",
    match_option = "INTERSECT",
    search_radius = None,
    distance_field_name = None,
    match_fields = None
)


<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\raw\\collisions1'>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.3. Analysis Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete all Old Analysis Feature Classes</h3>

Loop through all analysis data feature dataset and delete all feature classes

In [42]:
for f in arcpy.ListFeatureClasses(feature_dataset = "analysis"):
    print(f"Deleting {f}...")
    arcpy.Delete_management(f)
    print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Create Major Roads</h3>

Separate the primary and secondary roads from the local roads

In [43]:
# Output feature class for the major roads
roadsMajor = os.path.join(gdbAnalysisData, "roadsMajor")

# Select the major (primary and secondary) roads from the roads feature class
arcpy.analysis.Select(
    in_features = roads,
    out_feature_class = roadsMajor,
    where_clause = "roadCat = 'Primary' Or roadCat = 'Secondary'"
)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:24:19 PM
Succeeded at Wednesday, May 7, 2025 12:24:20 PM (Elapsed Time: 1.22 seconds)


Add feature class alias for the major roads feature class

In [44]:
# Define the major roads layer alias and modify the feature class alias
roadsMajorAlias = "OCSWITRS Major Roads"
arcpy.AlterAliasName(roadsMajor, roadsMajorAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:24:19 PM
Succeeded at Wednesday, May 7, 2025 12:24:20 PM (Elapsed Time: 1.22 seconds)


Obtain the list of fields for the major roads feature class

In [45]:
roadsMajorFields = [f.name for f in arcpy.ListFields(roadsMajor)] # roadsMajor field list

# Field Aliases for the major roads feature class
for f in arcpy.ListFields(roadsMajor):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
placeName (Road City)
roadName (Road Name)
roadId (Road ID)
roadCat (Road Category)
roadLength (Road Length (in miles))
roadsPrimary (Number of Primary Road Segments)
roadsSecondary (Number of Secondary Road Segments)
roadsLocal (Number of Local Road Segments)
roadLengthMean (Mean Road Length)
roadLengthSum (Total Road Length)
Shape_Length (Shape_Length)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Create Major Road Buffers</h3>

Create road buffers for the primary and secondary roads

In [46]:
# Output feature class for the major roads buffers
roadsMajorBuffers = os.path.join(gdbAnalysisData, "roadsMajorBuffers")

# Buffer the major roads feature class by 250 meters (on each side)
arcpy.analysis.Buffer(
    in_features = roadsMajor,
    out_feature_class = roadsMajorBuffers,
    buffer_distance_or_field = "250 Meters",
    line_side = "FULL",
    line_end_type = "FLAT",
    dissolve_option = "NONE",
    dissolve_field = None,
    method = "PLANAR"
)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:24:48 PM
Succeeded at Wednesday, May 7, 2025 12:24:50 PM (Elapsed Time: 2.07 seconds)


Add feature class alias for the major road buffers feature class

In [47]:
# Define the major roads buffers layer alias and modify the feature class alias
roadsMajorBuffersAlias = "OCSWITRS Major Roads Buffers"
arcpy.AlterAliasName(roadsMajorBuffers, roadsMajorBuffersAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:24:48 PM
Succeeded at Wednesday, May 7, 2025 12:24:50 PM (Elapsed Time: 2.07 seconds)


Obtain the list of fields for the major road buffers feature class

In [48]:
roadsMajorBuffersFields = [f.name for f in arcpy.ListFields(roadsMajorBuffers)] # roadsMajorBuffers field list

# Field Aliases for the major roads buffers feature class
for f in arcpy.ListFields(roadsMajorBuffers):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
placeName (Road City)
roadName (Road Name)
roadId (Road ID)
roadCat (Road Category)
roadLength (Road Length (in miles))
roadsPrimary (Number of Primary Road Segments)
roadsSecondary (Number of Secondary Road Segments)
roadsLocal (Number of Local Road Segments)
roadLengthMean (Mean Road Length)
roadLengthSum (Total Road Length)
BUFF_DIST (BUFF_DIST)
ORIG_FID (ORIG_FID)
Shape_Length (Shape_Length)
Shape_Area (Shape_Area)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize Major Road Buffers</h3>

Create a summary for each of the road buffers that contains statistics and counts of crash collision data

In [49]:
# Output feature class for the summarized major roads buffers
roadsMajorBuffersSum = os.path.join(gdbAnalysisData, "roadsMajorBuffersSum")

# Summarize the major roads buffers feature class by key crashes attributes
arcpy.analysis.SummarizeWithin(
    in_polygons = roadsMajorBuffers,
    in_sum_features = crashes,
    out_feature_class = roadsMajorBuffersSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields = [
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:25:06 PM
Succeeded at Wednesday, May 7, 2025 12:25:34 PM (Elapsed Time: 27.34 seconds)


Add feature class alias for the summarized major road buffers feature class

In [50]:
# Define the major roads buffers summary layer alias and modify the feature class alias
roadsMajorBuffersSumAlias = "OCSWITRS Major Roads Buffers Summary"
arcpy.AlterAliasName(roadsMajorBuffersSum, roadsMajorBuffersSumAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:25:06 PM
Succeeded at Wednesday, May 7, 2025 12:25:34 PM (Elapsed Time: 27.34 seconds)


Obtain the fields for the summarized major road buffers feature class

In [51]:
roadsMajorBuffersSumFields = [f.name for f in arcpy.ListFields(roadsMajorBuffersSum)] # roadsMajorBuffersSum field list

# Field Aliases for the major roads buffers summary feature class
for f in arcpy.ListFields(roadsMajorBuffersSum):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
placeName (Road City)
roadName (Road Name)
roadId (Road ID)
roadCat (Road Category)
roadLength (Road Length (in miles))
roadsPrimary (Number of Primary Road Segments)
roadsSecondary (Number of Secondary Road Segments)
roadsLocal (Number of Local Road Segments)
roadLengthMean (Mean Road Length)
roadLengthSum (Total Road Length)
BUFF_DIST (BUFF_DIST)
ORIG_FID (ORIG_FID)
Shape_Length (Shape_Length)
Shape_Area (Shape_Area)
sum_crashTag (Sum Crash Tag)
sum_partyCount (Sum Party Count)
sum_victimCount (Sum Victim Count)
sum_numberKilled (Sum Killed Victims)
sum_numberInj (Sum Injured Victims)
sum_countSevereInj (Sum Severe Injury Count)
sum_countVisibleInj (Sum Visible Injury Count)
sum_countComplaintPain (Sum Complaint of Pain Injury Count)
sum_countCarKilled (Sum Number of Killed Car Victims)
sum_countCarInj (Sum Number of Injured Car Victims)
sum_countPedKilled (Sum Pedestrian Killed Count)
sum_countPedInj (Sum Pedestrian Injury Count)
sum_countBicKilled 

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Points 1,000 ft along major road lines</h3>

Generate points every 1,000 feet along the major road lines

In [52]:
# Create a path for the new summarized major road buffers feature class
roadsMajorPointsAlongLines = os.path.join(gdbAnalysisData, "roadsMajorPointsAlongLines")

arcpy.management.GeneratePointsAlongLines(
    Input_Features = roadsMajor,
    Output_Feature_Class = roadsMajorPointsAlongLines,
    Point_Placement = "DISTANCE",
    Distance = "1000 Feet",
    Percentage = None,
    Include_End_Points = "NO_END_POINTS",
    Add_Chainage_Fields = "NO_CHAINAGE",
    Distance_Field=None,
    Distance_Method = "PLANAR"
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\analysis\\roadsMajorPointsAlongLines'>

Add feature class alias for the points along major road lines feature class

In [53]:
# Define the major roads points along lines layer alias and modify the feature class alias
roadsMajorPointsAlongLinesAlias = "OCSWITRS Major Roads Points Along Lines"
arcpy.AlterAliasName(roadsMajorPointsAlongLines, roadsMajorPointsAlongLinesAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:26:12 PM
Succeeded at Wednesday, May 7, 2025 12:26:16 PM (Elapsed Time: 4.18 seconds)


Obtain the fields for the points along major road lines feature class

In [54]:
roadsMajorPointsAlongLinesFields = [f.name for f in arcpy.ListFields(roadsMajorPointsAlongLines)] # roadsMajorPointsAlongLines field list

# Field Aliases for the major roads points along lines feature class
for f in arcpy.ListFields(roadsMajorPointsAlongLines):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
ORIG_FID (ORIG_FID)
placeName (Road City)
roadName (Road Name)
roadId (Road ID)
roadCat (Road Category)
roadLength (Road Length (in miles))
roadsPrimary (Number of Primary Road Segments)
roadsSecondary (Number of Secondary Road Segments)
roadsLocal (Number of Local Road Segments)
roadLengthMean (Mean Road Length)
roadLengthSum (Total Road Length)
Shape_Length (Shape_Length)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Split Road Segments 1,000ft apart</h3>

Split road segments at the points (1,000 feet apart)

In [55]:
# Create a path for the new split major roads feature class
roadsMajorSplit = os.path.join(gdbAnalysisData, "roadsMajorSplit")

# Split the major roads at the points along the lines
arcpy.management.SplitLineAtPoint(
    in_features = roadsMajor,
    point_features = roadsMajorPointsAlongLines,
    out_feature_class = roadsMajorSplit,
    search_radius = "1000 Feet"
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\analysis\\roadsMajorSplit'>

Add feature class alias for the split road segments feature class

In [56]:
# Define the major roads split layer alias and modify the feature class alias
roadsMajorSplitAlias = "OCSWITRS Major Roads Split"
arcpy.AlterAliasName(roadsMajorSplit, roadsMajorSplitAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:26:28 PM
Succeeded at Wednesday, May 7, 2025 12:26:37 PM (Elapsed Time: 8.40 seconds)


Obtain the fields for the split road segments feature class

In [57]:
roadsMajorSplitFields = [f.name for f in arcpy.ListFields(roadsMajorSplit)] # roadsMajorSplit field list

# Field Aliases for the major roads split feature class
for f in arcpy.ListFields(roadsMajorSplit):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
placeName (Road City)
roadName (Road Name)
roadId (Road ID)
roadCat (Road Category)
roadLength (Road Length (in miles))
roadsPrimary (Number of Primary Road Segments)
roadsSecondary (Number of Secondary Road Segments)
roadsLocal (Number of Local Road Segments)
roadLengthMean (Mean Road Length)
roadLengthSum (Total Road Length)
ORIG_FID (ORIG_FID)
ORIG_SEQ (ORIG_SEQ)
Shape_Length (Shape_Length)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Buffers 500ft around road segments</h3>

Create buffers (500 ft) around the road segments (1,000 feet)

In [58]:
# Create a path for the new split major roads feature class
roadsMajorSplitBuffer = os.path.join(gdbAnalysisData, "roadsMajorSplitBuffer")

# Buffer the split major roads by 500 feet
arcpy.analysis.Buffer(
    in_features = roadsMajorSplit,
    out_feature_class = roadsMajorSplitBuffer,
    buffer_distance_or_field = "500 Feet",
    line_side = "FULL",
    line_end_type = "FLAT",
    dissolve_option = "NONE",
    dissolve_field = None,
    method = "PLANAR"
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\analysis\\roadsMajorSplitBuffer'>

Add feature class alias for the road segment buffers feature class

In [59]:
# Define the major roads split buffer layer alias and modify the feature class alias
roadsMajorSplitBufferAlias = "OCSWITRS Major Roads Split Buffer"
arcpy.AlterAliasName(roadsMajorSplitBuffer, roadsMajorSplitBufferAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:26:48 PM
WARNING 000636: Skipping feature 64 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 344 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 345 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 625 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 1294 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 1295 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 1296 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 1297 because a negative or very small distance resulted in no geometry.
WARNING 000636: Skipping feature 1308 because a negative or very small distance resulted in no geometry.
WARNING 00063

Obtain the fields for the road segment buffers feature class

In [60]:
roadsMajorSplitBufferFields = [f.name for f in arcpy.ListFields(roadsMajorSplitBuffer)] # roadsMajorSplitBuffer field list

# Field Aliases for the major roads split buffer feature class
for f in arcpy.ListFields(roadsMajorSplitBuffer):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
placeName (Road City)
roadName (Road Name)
roadId (Road ID)
roadCat (Road Category)
roadLength (Road Length (in miles))
roadsPrimary (Number of Primary Road Segments)
roadsSecondary (Number of Secondary Road Segments)
roadsLocal (Number of Local Road Segments)
roadLengthMean (Mean Road Length)
roadLengthSum (Total Road Length)
ORIG_FID (ORIG_FID)
ORIG_SEQ (ORIG_SEQ)
BUFF_DIST (BUFF_DIST)
Shape_Length (Shape_Length)
Shape_Area (Shape_Area)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize road segments buffers</h3>

Summarize the crash collision data for each of the road segments

In [61]:
# Create a path for the new summarized major road buffers feature class
roadsMajorSplitBufferSum = os.path.join(gdbAnalysisData, "roadsMajorSplitBufferSum")

# Summarize the data within the major road buffers from the crashes data
arcpy.analysis.SummarizeWithin(
    in_polygons = roadsMajorSplitBuffer,
    in_sum_features = crashes,
    out_feature_class = roadsMajorSplitBufferSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields=[
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\analysis\\roadsMajorSplitBufferSum'>

Add feature class alias for the summarized road segments buffers feature class

In [62]:
# Define the major roads split buffer summary layer alias and modify the feature class alias
roadsMajorSplitBufferSumAlias = "OCSWITRS Major Roads Split Buffer Summary"
arcpy.AlterAliasName(roadsMajorSplitBufferSum, roadsMajorSplitBufferSumAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:27:09 PM
Succeeded at Wednesday, May 7, 2025 12:27:44 PM (Elapsed Time: 35.19 seconds)


Obtain the fields for the summarized road segments buffers feature class

In [63]:
roadsMajorSplitBufferSumFields = [f.name for f in arcpy.ListFields(roadsMajorSplitBufferSum)] # roadsMajorSplitBufferSum field list

# Field Aliases for the major roads split buffer summary feature class
for f in arcpy.ListFields(roadsMajorSplitBufferSum):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
placeName (Road City)
roadName (Road Name)
roadId (Road ID)
roadCat (Road Category)
roadLength (Road Length (in miles))
roadsPrimary (Number of Primary Road Segments)
roadsSecondary (Number of Secondary Road Segments)
roadsLocal (Number of Local Road Segments)
roadLengthMean (Mean Road Length)
roadLengthSum (Total Road Length)
ORIG_FID (ORIG_FID)
ORIG_SEQ (ORIG_SEQ)
BUFF_DIST (BUFF_DIST)
Shape_Length (Shape_Length)
Shape_Area (Shape_Area)
sum_crashTag (Sum Crash Tag)
sum_partyCount (Sum Party Count)
sum_victimCount (Sum Victim Count)
sum_numberKilled (Sum Killed Victims)
sum_numberInj (Sum Injured Victims)
sum_countSevereInj (Sum Severe Injury Count)
sum_countVisibleInj (Sum Visible Injury Count)
sum_countComplaintPain (Sum Complaint of Pain Injury Count)
sum_countCarKilled (Sum Number of Killed Car Victims)
sum_countCarInj (Sum Number of Injured Car Victims)
sum_countPedKilled (Sum Pedestrian Killed Count)
sum_countPedInj (Sum Pedestrian Injury Count)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize Census Blocks</h3>

Create a summary for each of the Census blocks that contains statistics and counts of crash collision data

In [65]:
# Create a path for the new summarized US 2020 Census Blocks feature class
blocksSum = os.path.join(gdbAnalysisData, "blocksSum")

arcpy.analysis.SummarizeWithin(
    in_polygons = blocks,
    in_sum_features = crashes,
    out_feature_class = blocksSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields = [
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\analysis\\blocksSum'>

Add feature class alias for the summarized Census blocks feature class

In [66]:
# Define the US 2020 Census Blocks summary layer alias and modify the feature class alias
blocksSumAlias = "OCSWITRS Census Blocks Summary"
arcpy.AlterAliasName(blocksSum, blocksSumAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:28:51 PM
Succeeded at Wednesday, May 7, 2025 12:29:34 PM (Elapsed Time: 42.92 seconds)


Obtain the fields for the summarized Census blocks feature class

In [68]:
blocksSumFields = [f.name for f in arcpy.ListFields(blocksSum)] # cenBlocksSum field list

# Field Aliases for the US 2020 Census Blocks summary feature class
for f in arcpy.ListFields(blocksSum):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
GEOID (GeoID)
SUFFIX (Block suffix)
NAME (Block name)
areaLand (Land area)
areaWater (Water area)
latitudeIn (Block latitude)
longitudeIn (Block longitude)
State (State)
County (County)
populationTotal (Total Population)
housingUnitsTotal (Total Housing Units)
householdsTotal (Total Households)
housingUnitsVacant (Vacant Housing Units)
housingUnitsMortgage (Housing Units with Mortgage)
housingUnitsOwned (Housing Units Owned)
housingUnitsRenterOccupied (Renter Occupied Housing Units)
vacancyRate (Vacancy Rate)
housingUnitsOwnerOccupied (Owner Occupied Housing Units)
homeownershipRate (Homeownership Rate)
populationDensity (Population Density)
housingDensity (Housing Density)
Shape_Leng (Shape_Leng)
Shape_Length (Shape_Length)
Shape_Area (Shape_Area)
sum_crashTag (Sum Crash Tag)
sum_partyCount (Sum Party Count)
sum_victimCount (Sum Victim Count)
sum_numberKilled (Sum Killed Victims)
sum_numberInj (Sum Injured Victims)
sum_countSevereInj (Sum Severe Injur

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Summarize Cities</h3>

Create a summary for each of the cities that contains statistics and counts of crash collision data

In [69]:
# Create a path for the new summarized US 2020 Census Blocks feature class
citiesSum = os.path.join(gdbAnalysisData, "citiesSum")

# Summarize the data within the cities from the crashes data
arcpy.analysis.SummarizeWithin(
    in_polygons = cities,
    in_sum_features = crashes,
    out_feature_class = citiesSum,
    keep_all_polygons = "KEEP_ALL",
    sum_fields=[
        ["crashTag", "Sum"], ["partyCount", "Sum"], ["victimCount", "Sum"], ["numberKilled", "Sum"], 
        ["numberInj", "Sum"], ["countSevereInj", "Sum"], ["countVisibleInj", "Sum"],
        ["countComplaintPain", "Sum"], ["countCarKilled", "Sum"], ["countCarInj", "Sum"],
        ["countPedKilled", "Sum"], ["countPedInj", "Sum"], ["countBicKilled", "Sum"],
        ["countBicInj", "Sum"], ["countMcKilled", "Sum"], ["countMcInj", "Sum"],
        ["collSeverityNum", "Mean"], ["collSeverityRankNum", "Mean"]
    ]
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\analysis\\citiesSum'>

Add feature class alias for the summarized cities feature class

In [70]:
# Define the cities summary layer alias and modify the feature class alias
citiesSumAlias = "OCSWITRS Cities Summary"
arcpy.AlterAliasName(citiesSum, citiesSumAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:30:37 PM
Succeeded at Wednesday, May 7, 2025 12:31:06 PM (Elapsed Time: 29.71 seconds)


Obtain the fields for the summarized cities feature class

In [71]:
citiesSumFields = [f.name for f in arcpy.ListFields(citiesSum)] # citiesSum field list

# Field Aliases for the cities summary feature class
for f in arcpy.ListFields(citiesSum):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
city (City)
placeType (Place Type)
cityAreaSqmi (City Area)
cityPopTotal (City Total Population)
cityHouTotal (City Total Housing Units)
cityPopDens (City Population Density)
cityHouDens (City Housing Density)
cityPopAsian (City Asian Population)
cityPopBlack (City Black Population)
cityPopHispanic (City Hispanic Population)
cityPopWhite (City White Population)
cityVehicles (Number of Commuting Vehicles)
cityTravelTime (Aggregate Travel Time to Work)
cityMeanTravelTime (Mean Travel Time to work Per Vehicle)
Shape_Length (Shape_Length)
Shape_Area (Shape_Area)
sum_crashTag (Sum Crash Tag)
sum_partyCount (Sum Party Count)
sum_victimCount (Sum Victim Count)
sum_numberKilled (Sum Killed Victims)
sum_numberInj (Sum Injured Victims)
sum_countSevereInj (Sum Severe Injury Count)
sum_countVisibleInj (Sum Visible Injury Count)
sum_countComplaintPain (Sum Complaint of Pain Injury Count)
sum_countCarKilled (Sum Number of Killed Car Victims)
sum_countCarInj (Sum Num

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes within 500ft from Major Roads</h3>

Select all crashes that are within 500 ft of the major roads

In [72]:
# Create a path for the new feature class
crashes500ftFromMajorRoads = os.path.join(gdbAnalysisData, "crashes500ftFromMajorRoads")

# Select the crashes within 500 feet of the major roads and store it in a temporary layer
tempLyr = arcpy.management.SelectLayerByLocation(
    in_layer = crashes,
    select_features = roadsMajor,
    search_distance = "500 Feet",
    selection_type = "NEW_SELECTION",
    invert_spatial_relationship = "NOT_INVERT"
)

# Export the selected crashes to a new feature class
arcpy.conversion.ExportFeatures(
    in_features = tempLyr,
    out_features = crashes500ftFromMajorRoads,
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
)
print(arcpy.GetMessages())

# Delete the temporary layer
arcpy.management.Delete(tempLyr)

Start Time: Wednesday, May 7, 2025 12:31:17 PM
Succeeded at Wednesday, May 7, 2025 12:31:27 PM (Elapsed Time: 9.46 seconds)


<Result 'true'>

Add feature class alias for the crashes within 500 ft from major roads feature class

In [73]:
# Define the crashes 500 feet from major roads layer alias and modify the feature class alias
crashes500ftFromMajorRoadsAlias = "OCSWITRS Crashes 500 Feet from Major Roads"
arcpy.AlterAliasName(crashes500ftFromMajorRoads, crashes500ftFromMajorRoadsAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 12:31:27 PM
Succeeded at Wednesday, May 7, 2025 12:31:27 PM (Elapsed Time: 0.02 seconds)


Obtain the fields for the crashes within 500 ft from major roads feature class

In [74]:
crashes500ftFromMajorRoadsFields = [f.name for f in arcpy.ListFields(crashes500ftFromMajorRoads)] # crashes500ftFromMajorRoads field list

# Field Aliases for the crashes 500 feet from major roads feature class
for f in arcpy.ListFields(crashes500ftFromMajorRoads):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
caseId (Case ID)
cid (Crash ID)
crashTag (Crash Tag)
crashesCaseTag (Crashes Tag)
crashesCidCount (Total Collisions in Crashes Dataset)
city (City)
dateDatetime (Crash Date and Time)
dateYear (Crash Year)
dateQuarter (Crash Quarter)
dateMonth (Crash Month)
dateWeek (Crash Week)
dateDay (Crash Day)
dateProcess (Process Date)
dtYear (Crash Year)
dtQuarter (Crash Quarter)
dtMonth (Crash Month)
dtYearWeek (Crash Year Week)
dtWeekDay (Crash Weekday)
dtMonthDay (Crash Month Day)
dtYearDay (Crash Year Day)
dtHour (Crash Hour)
dtMinute (Crash Minute)
dtDst (Crash Daylight Saving Time)
dtZone (Crash Time Zone)
collDate (Collision Date Reported)
collTime (Collision Time Reported)
accidentYear (Crash Year)
processDate (Processing Date)
collTimeIntervals (Crash Time Intervals)
rushHours (Rush Hours)
rushHoursBin (Rush Hours Binary)
collSeverity (Collision Severity)
collSeverityNum (Collision Severity Numeric)
collSeverityBin (Collision Severity Binary)
collSeverit

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Crashes (Collision Severity) Exploratory Regression</h3>

Generate a collision severity binary indicator to crashes dataset

In [75]:
# Add collision severity binary indicator to crashes
arcpy.management.CalculateField(
    in_table = crashes,
    field = "severityBin",
    expression = "sevbin(!collSeverityBin!)",
    expression_type = "PYTHON3",
    code_block = """def sevbin(x):
    if x == "Severe or fatal":
        return 1
    elif x == "None, minor or pain":
        return 0""",
    field_type = "SHORT",
    enforce_domains = "NO_ENFORCE_DOMAINS"
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\raw\\crashes'>

Add a field alias for the collision severity binary indicator

In [76]:
arcpy.management.AlterField(
    in_table = crashes,
    field = "severityBin",
    new_field_alias = "Severity Binary"
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\raw\\crashes'>

Perform exploratory regression to predict the binary severity bin

In [ ]:
arcpy.stats.ExploratoryRegression(
    Input_Features = crashes,
    Dependent_Variable = "severityBin",
    Candidate_Explanatory_Variables = "accidentYear;collSeverityNum;collSeverityRankNum;partyCount;victimCount;numberKilled;numberInj;countSevereInj;countVisibleInj;countComplaintPain;countCarKilled;countCarInj;countPedKilled;countPedInj;countBicKilled;countBicInj;countMcKilled;countMcInj",
    Weights_Matrix_File = None,
    Output_Report_File = None,
    Output_Results_Table = None,
    Maximum_Number_of_Explanatory_Variables = 5,
    Minimum_Number_of_Explanatory_Variables = 1,
    Minimum_Acceptable_Adj_R_Squared = 0.5,
    Maximum_Coefficient_p_value_Cutoff = 0.05,
    Maximum_VIF_Value_Cutoff = 7.5,
    Minimum_Acceptable_Jarque_Bera_p_value = 0.1,
    Minimum_Acceptable_Spatial_Autocorrelation_p_value = 0.1
)

<Result ''>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [78]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.4. Hotspot Data Feature Classes</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Delete all Old Hotspot Feature Classes</h3>

Loop through all hotspot data feature dataset and delete all feature classes

In [79]:
for f in arcpy.ListFeatureClasses(feature_dataset = "hotspots"):
    print(f"Deleting {f}...")
    arcpy.Delete_management(f)
    print(arcpy.GetMessages())

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Create Hot Spots (Crashes, Collision Severity)</h3>

Create a hot spot analysis for the crash collision data (collision severity)

In [80]:
# Create a path for the new crashes hot spots feature class
crashesHotspots = os.path.join(gdbHotspotData, "crashesHotspots")

# Create hot spots points
arcpy.stats.HotSpots(
    Input_Feature_Class = crashes,
    Input_Field = "collSeverityNum",
    Output_Feature_Class = crashesHotspots,
    Conceptualization_of_Spatial_Relationships = "FIXED_DISTANCE_BAND",
    Distance_Method = "EUCLIDEAN_DISTANCE",
    Standardization = "ROW",
    Distance_Band_or_Threshold_Distance = None,
    Self_Potential_Field = None,
    Weights_Matrix_File = None,
    Apply_False_Discovery_Rate__FDR__Correction = "NO_FDR",
    number_of_neighbors = None
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\hotspots\\crashesHotspots'>

Add feature class alias for the hot spots (crashes, collision severity) feature class

In [81]:
# Define the crashes hot spots layer alias and modify the feature class alias
crashesHotspotsAlias = "OCSWITRS Crashes Hot Spots"
arcpy.AlterAliasName(crashesHotspots, crashesHotspotsAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 1:00:44 PM
WARNING 000853: The default neighborhood search threshold was 5800.7226 meters.
WARNING 001420: The number of neighbors for at least one feature exceeds 1000, you may want to change your conceptualization of spatial relationships to avoid the potential of running out of memory.
WARNING 001422: Features with number of neighbors exceeding warning value (only includes first 30): OBJECTID = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30.
Succeeded at Wednesday, May 7, 2025 1:07:13 PM (Elapsed Time: 6 minutes 28 seconds)


Obtain the fields for the hot spots (crashes, collision severity) feature class

In [82]:
crashesHotspotsFields = [f.name for f in arcpy.ListFields(crashesHotspots)] # crashesHotspots field list

# Field Aliases for the crashes hot spots feature class
for f in arcpy.ListFields(crashesHotspots):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
SOURCE_ID (SOURCE_ID)
collSeverityNum (Collision Severity Numeric)
GiZScore (GiZScore Fixed 5800)
GiPValue (GiPValue Fixed 5800)
NNeighbors (NNeighbors Fixed 5800)
Gi_Bin (Gi_Bin Fixed 5800)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Optimized Hot Spots (Crashes, Collision Severity, 1,000 m)</h3>

Optimized hot spot analysis for the crash collision data (collision severity)

In [83]:
# Create a path for the new optimized crashes hot spots feature class
crashesOptimizedHotspots = os.path.join(gdbHotspotData, "crashesOptimizedHotspots")

# Perform Optimized Hot Spot Analysis on the crashes data
arcpy.stats.OptimizedHotSpotAnalysis(
    Input_Features = crashes,
    Output_Features = crashesOptimizedHotspots,
    Analysis_Field = "collSeverityNum",
    Incident_Data_Aggregation_Method = "COUNT_INCIDENTS_WITHIN_FISHNET_POLYGONS",
    Bounding_Polygons_Defining_Where_Incidents_Are_Possible = None,
    Polygons_For_Aggregating_Incidents_Into_Counts = None,
    Density_Surface = None,
    Cell_Size = None,
    Distance_Band = "1000 Meters"
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\hotspots\\crashesOptimizedHotspots'>

Add feature class alias for the optimized hot spots (crashes, collision severity, 1,000 m) feature class

In [84]:
# Define the optimized crashes hot spots layer alias and modify the feature class alias
crashesOptimizedHotspotsAlias = "OCSWITRS Crashes Optimized Hot Spots"
arcpy.AlterAliasName(crashesOptimizedHotspots, crashesOptimizedHotspotsAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 1:24:12 PM

************************** Initial Data Assessment ***************************
Making sure there are enough weighted features for analysis....
    - There are 170195 valid input features.

Evaluating the Analysis Field values....
    - COLLSEVERITYNUM Properties:
        Min:       1.0000
        Max:       4.0000
        Mean:      1.4658
        Std. Dev.: 0.6586
Looking for locational outliers....
    - There were 2529 outlier locations; these will not be used to compute the optimal fixed distance band.

***************************** Scale of Analysis ******************************
    - The Neighborhood Distance used was 1000 meters.

***************************** Hot Spot Analysis ******************************
Finding statistically significant clusters of high and low Collision Severity Numeric values....
WARNING 001420: The number of neighbors for at least one feature exceeds 1000, you may want to change your conceptualization of sp

Obtain the fields for the optimized hot spots (crashes, collision severity, 1,000 m) feature class

In [85]:
crashesOptimizedHotspotsFields = [f.name for f in arcpy.ListFields(crashesOptimizedHotspots)] # crashesOptimizedHotspots field list

# Field Aliases for the optimized crashes hot spots feature class
for f in arcpy.ListFields(crashesOptimizedHotspots):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
SOURCE_ID (SOURCE_ID)
collSeverityNum (Collision Severity Numeric)
GiZScore (GiZScore Fixed 1000)
GiPValue (GiPValue Fixed 1000)
NNeighbors (NNeighbors Fixed 1000)
Gi_Bin (Gi_Bin Fixed 1000_FDR)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Crashes, 100m bins, 1km neighbors)</h3>

Find hot spots for the crash collision data using 100 m bins and 1 km neighborhood radius (328 ft/ 0.621 mi)

In [86]:
# Create a path for the new crashes find hot spots feature class
crashesFindHotspots100m1km = os.path.join(gdbHotspotData, "crashesFindHotspots100m1km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes,
    out_feature_class = crashesFindHotspots100m1km,
    bin_size = "100 Meters",
    neighborhood_size = "1 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\hotspots\\crashesFindHotspots100m1km'>

Add feature class alias for the hot spots (crashes, 100m bins, 1km neighbors) feature class

In [87]:
# Define the crashes find hot spots layer alias and modify the feature class alias
crashesFindHotspots100m1kmAlias = "OCSWITRS Crashes Find Hot Spots 100m 1km"
arcpy.AlterAliasName(crashesFindHotspots100m1km, crashesFindHotspots100m1kmAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 1:25:39 PM
Started Spark Job 1 with 16 tasks.
Spark Job 1 completed in 4.1 seconds.
Started Spark Job 2 with 32 tasks.
Spark Job 2 completed in 4.6 seconds.
Started Spark Job 3 with 24 tasks.
Spark Job 3 completed in 0.8 seconds.
Succeeded at Wednesday, May 7, 2025 1:26:05 PM (Elapsed Time: 26.04 seconds)


Obtain the fields for the hot spots (crashes, 100m bins, 1km neighbors) feature class

In [88]:
crashesFindHotspots100m1kmFields = [f.name for f in arcpy.ListFields(crashesFindHotspots100m1km)] # crashesFindHotspots100m1km field list

# Field Aliases for the crashes find hot spots feature class
for f in arcpy.ListFields(crashesFindHotspots100m1km):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
value (value)
GiZScore (GiZScore)
GiPValue (GiPValue)
Gi_Bin (Gi_Bin)
SHAPE (SHAPE)
SHAPE_Length (SHAPE_Length)
SHAPE_Area (SHAPE_Area)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Crashes, 150m bins, 2km neighbors)</h3>

Find hot spots for the crash collision data using 150 m bins and 2 km neighborhood radius (492 ft/ 1.24 mi)

In [89]:
# Create a path for the new crashes find hot spots feature class
crashesFindHotspots150m2km = os.path.join(gdbHotspotData, "crashesFindHotspots150m2km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes,
    out_feature_class = crashesFindHotspots150m2km,
    bin_size = "150 Meters",
    neighborhood_size = "2 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\hotspots\\crashesFindHotspots150m2km'>

Add feature class alias for the hot spots (crashes, 150m bins, 2km neighbors) feature class

In [90]:
# Define the crashes find hot spots layer alias and modify the feature class alias
crashesFindHotspots150m2kmAlias = "OCSWITRS Crashes Find Hot Spots 150m 2km"
arcpy.AlterAliasName(crashesFindHotspots150m2km, crashesFindHotspots150m2kmAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 1:26:19 PM
Started Spark Job 1 with 16 tasks.
Spark Job 1 completed in 3.3 seconds.
Started Spark Job 2 with 32 tasks.
Spark Job 2 completed in 3.7 seconds.
Started Spark Job 3 with 24 tasks.
Spark Job 3 completed in 0.8 seconds.
Succeeded at Wednesday, May 7, 2025 1:26:31 PM (Elapsed Time: 12.54 seconds)


Obtain the fields for the hot spots (crashes, 150m bins, 2km neighbors) feature class

In [91]:
crashesFindHotspots150m2kmFields = [f.name for f in arcpy.ListFields(crashesFindHotspots150m2km)] # crashesFindHotspots150m2km field list

# Field Aliases for the crashes find hot spots feature class
for f in arcpy.ListFields(crashesFindHotspots150m2km):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
value (value)
GiZScore (GiZScore)
GiPValue (GiPValue)
Gi_Bin (Gi_Bin)
SHAPE (SHAPE)
SHAPE_Length (SHAPE_Length)
SHAPE_Area (SHAPE_Area)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Crashes, 100m bins, 5km neighbors)</h3>

Find hot spots for the crash collision data using 100 m bins and 5 km neighborhood radius (328 ft/ 3.11 mi)

In [92]:
# Create a path for the new crashes find hot spots feature class
crashesFindHotspots100m5km = os.path.join(gdbHotspotData, "crashesFindHotspots100m5km")

# Find the hot spots within the crashes data
arcpy.gapro.FindHotSpots(
    point_layer = crashes,
    out_feature_class = crashesFindHotspots100m5km,
    bin_size = "100 Meters",
    neighborhood_size = "5 Kilometers",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference = None
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\hotspots\\crashesFindHotspots100m5km'>

Add feature class alias for the hot spots (crashes, 100m bins, 5km neighbors) feature class

In [93]:
# Define the crashes find hot spots layer alias and modify the feature class alias
crashesFindHotspots100m5kmAlias = "OCSWITRS Crashes Find Hot Spots 100m 5km"
arcpy.AlterAliasName(crashesFindHotspots100m5km, crashesFindHotspots100m5kmAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 1:26:43 PM
Started Spark Job 1 with 16 tasks.
Spark Job 1 completed in 3.1 seconds.
Started Spark Job 2 with 32 tasks.
Spark Job 2 completed in 3.4 seconds.
Started Spark Job 3 with 24 tasks.
Spark Job 3 completed in 5.1 seconds.
Succeeded at Wednesday, May 7, 2025 1:27:00 PM (Elapsed Time: 16.40 seconds)


Obtain the fields for the hot spots (crashes, 100m bins, 5km neighbors) feature class

In [94]:
crashesFindHotspots100m5kmFields = [f.name for f in arcpy.ListFields(crashesFindHotspots100m5km)] # crashesFindHotspots100m5km field list

# Field Aliases for the crashes find hot spots feature class
for f in arcpy.ListFields(crashesFindHotspots100m5km):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
value (value)
GiZScore (GiZScore)
GiPValue (GiPValue)
Gi_Bin (Gi_Bin)
SHAPE (SHAPE)
SHAPE_Length (SHAPE_Length)
SHAPE_Area (SHAPE_Area)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Hot Spots (Proximity to Major Roads, 500ft)</h3>

Hot spot points within 500 feet of major roads

In [95]:
# Create a path for the new hot spots within 500 mt from major roads feature class
crashesHotspots500ftFromMajorRoads = os.path.join(gdbHotspotData, "crashesHotspots500ftFromMajorRoads")

arcpy.stats.HotSpots(
    Input_Feature_Class = crashes500ftFromMajorRoads,
    Input_Field="collSeverityNum",
    Output_Feature_Class = crashesHotspots500ftFromMajorRoads,
    Conceptualization_of_Spatial_Relationships = "FIXED_DISTANCE_BAND",
    Distance_Method = "EUCLIDEAN_DISTANCE",
    Standardization = "ROW",
    Distance_Band_or_Threshold_Distance = None,
    Self_Potential_Field = None,
    Weights_Matrix_File = None,
    Apply_False_Discovery_Rate__FDR__Correction = "NO_FDR",
    number_of_neighbors = None
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\hotspots\\crashesHotspots500ftFromMajorRoads'>

Add feature class alias for the hot spots (proximity to major roads, 500ft) feature class

In [96]:
# Define the crashes hot spots 500 feet from major roads layer alias and modify the feature class alias
crashesHotspots500ftFromMajorRoadsAlias = "OCSWITRS Crashes Hot Spots 500 Feet from Major Roads"
arcpy.AlterAliasName(crashesHotspots500ftFromMajorRoads, crashesHotspots500ftFromMajorRoadsAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 1:27:13 PM
WARNING 000853: The default neighborhood search threshold was 872.0926 meters.
WARNING 001420: The number of neighbors for at least one feature exceeds 1000, you may want to change your conceptualization of spatial relationships to avoid the potential of running out of memory.
WARNING 001422: Features with number of neighbors exceeding warning value (only includes first 30): OBJECTID = 133, 146, 303, 309, 430, 442, 500, 529, 534, 584, 694, 796, 849, 861, 889, 945, 1012, 1200, 1224, 1231, 1322, 1353, 1356, 1418, 1460, 1465, 1467, 1494, 1500, 1514.
Succeeded at Wednesday, May 7, 2025 1:27:31 PM (Elapsed Time: 17.87 seconds)


Obtain the fields for the hot spots (proximity to major roads, 500ft) feature class

In [97]:
crashesHotspots500ftFromMajorRoadsFields = [f.name for f in arcpy.ListFields(crashesHotspots500ftFromMajorRoads)] # crashesHotspots500ftFromMajorRoads field list

# Field Aliases for the crashes hot spots 500 feet from major roads feature class
for f in arcpy.ListFields(crashesHotspots500ftFromMajorRoads):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
Shape (Shape)
SOURCE_ID (SOURCE_ID)
collSeverityNum (Collision Severity Numeric)
GiZScore (GiZScore Fixed 872)
GiPValue (GiPValue Fixed 872)
NNeighbors (NNeighbors Fixed 872)
Gi_Bin (Gi_Bin Fixed 872)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Find Hot Spots (Proximity to Major Roads, 500ft)</h3>

Find hot spots within 500 feet from major roads

In [ ]:
# Create a path for the new hot spots within 500 ft from major roads feature class
crashesFindHotspots500ftMajorRoads500ft1mi = os.path.join(gdbHotspotData, "crashesFindHotspots500ftMajorRoads500ft1mi")

arcpy.gapro.FindHotSpots(
    point_layer = crashes500ftFromMajorRoads,
    out_feature_class = crashesFindHotspots500ftMajorRoads500ft1mi,
    bin_size = "500 Feet",
    neighborhood_size = "1 Miles",
    time_step_interval = None,
    time_step_alignment = "START_TIME",
    time_step_reference=None
)

<Result 'c:\\Users\\ktalexan\\OneDrive\\Documents\\GitHub\\OCSWITRS\\AGPSWITRS\\AGPSWITRS.gdb\\hotspots\\crashesFindHotspots500ftMajorRoads500ft1mi'>

Add feature class alias for the hot spots (proximity to major roads, 500ft) feature class

In [99]:
# Define the crashes find hot spots 500 feet from major roads layer alias and modify the feature class alias
crashesFindHotspots500ftMajorRoads500ft1miAlias = "OCSWITRS Crashes Find Hot Spots 500 Feet from Major Roads 500ft 1mi"
arcpy.AlterAliasName(crashesFindHotspots500ftMajorRoads500ft1mi, crashesFindHotspots500ftMajorRoads500ft1miAlias)
print(arcpy.GetMessages())

Start Time: Wednesday, May 7, 2025 1:27:43 PM
Started Spark Job 1 with 16 tasks.
Spark Job 1 completed in 1.5 seconds.
Started Spark Job 2 with 32 tasks.
Spark Job 2 completed in 1.8 seconds.
Started Spark Job 3 with 24 tasks.
Spark Job 3 completed in 0.1 seconds.
Succeeded at Wednesday, May 7, 2025 1:27:50 PM (Elapsed Time: 7.49 seconds)


Obtain the fields for the hot spots (proximity to major roads, 500ft) feature class

In [100]:
crashesFindHotspots500ftMajorRoads500ft1miFields = [f.name for f in arcpy.ListFields(crashesFindHotspots500ftMajorRoads500ft1mi)] # crashesFindHotspots500ftMajorRoads500ft1mi field list

# Field Aliases for the crashes find hot spots 500 feet from major roads feature class
for f in arcpy.ListFields(crashesFindHotspots500ftMajorRoads500ft1mi):
    print(f"{f.name} ({f.aliasName})")

OBJECTID (OBJECTID)
value (value)
GiZScore (GiZScore)
GiPValue (GiPValue)
Gi_Bin (Gi_Bin)
SHAPE (SHAPE)
SHAPE_Length (SHAPE_Length)
SHAPE_Area (SHAPE_Area)


<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">3. Geodatabase, Feature Dataset and Feature Class Metadata Processing</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.1. Project Geodatabase Metadata</h2>

Create a metadata object for the project geodatabase

In [101]:
# Define key metadata attributes for the AGPSWITRS geodatabase
mdoGdb = md.Metadata()
mdoGdb.title = "AGPSWITRS Historical Traffic Collisions"
mdoGdb.tags = "Orange County, California, OCSWITRS, Traffic, Traffic Conditions, Crashes, Collisions, Parties, Victims, Injuries, Fatalities, Hot Spots, Road Safety, Accidents, SWITRS, Transportation"
mdoGdb.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdYears})"
mdoGdb.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoGdb.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoGdb.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoGdb.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the geodatabase metadata object to the project geodatabase

In [ ]:
# Apply the metadata object to the project geodatabase
mdGdb = md.Metadata(gdbPath)
if not mdGdb.isReadOnly:
    mdGdb.copy(mdoGdb)
    mdGdb.save()
    print(f"Metadata updated for {gdbName} geodatabase.")

Metadata updated for AGPSWITRS.gdb geodatabase.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.2. Analysis Data Feature Dataset Metadata</h2>

Create a new metadata object for the analysis feature dataset

In [103]:
mdoAnalysis = md.Metadata()
mdoAnalysis.title = "OCSWITRS Traffic Collisions Analysis Dataset"
mdoAnalysis.tags = "Orange County, California, OCSWITRS, Traffic, Traffic Conditions, Crashes, Collisions, Parties, Victims, Injuries, Fatalities, Hot Spots, Road Safety, Accidents, SWITRS, Transportation"
mdoAnalysis.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdYears})"
mdoAnalysis.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoAnalysis.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoAnalysis.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoAnalysis.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the metadata object to the analysis feature dataset

In [104]:
# Apply the metadata object to the analysis feature dataset
mdAnalysis = md.Metadata(gdbAnalysisData)
if not mdAnalysis.isReadOnly:
    mdAnalysis.copy(mdoAnalysis)
    mdAnalysis.save()
    print(f"Metadata updated for the analysis feature dataset.")

Metadata updated for the analysis feature dataset.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.3. HotSpot Data Feature Dataset Metadata</h2>

Create a new metadata object for the hotspots feature dataset

In [105]:
mdoHotspots = md.Metadata()
mdoHotspots.title = "OCSWITRS Traffic Collisions Hotspots Dataset"
mdoHotspots.tags = "Orange County, California, OCSWITRS, Traffic, Traffic Conditions, Crashes, Collisions, Parties, Victims, Injuries, Fatalities, Hot Spots, Road Safety, Accidents, SWITRS, Transportation"
mdoHotspots.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdYears})"
mdoHotspots.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoHotspots.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoHotspots.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoHotspots.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the metadata object to the hotspots feature dataset

In [106]:
# Apply the metadata object to the hotspots feature dataset
mdHotspots = md.Metadata(gdbHotspotData)
if not mdHotspots.isReadOnly:
    mdHotspots.copy(mdoHotspots)
    mdHotspots.save()
    print(f"Metadata updated for the hotspots feature dataset.")

Metadata updated for the hotspots feature dataset.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.4. Raw Data Feature Dataset Metadata</h2>

Create a new metadata object for the raw feature dataset

In [107]:
mdoRaw = md.Metadata()
mdoRaw.title = "OCSWITRS Traffic Collisions Raw Data Dataset"
mdoRaw.tags = "Orange County, California, OCSWITRS, Traffic, Traffic Conditions, Crashes, Collisions, Parties, Victims, Injuries, Fatalities, Hot Spots, Road Safety, Accidents, SWITRS, Transportation"
mdoRaw.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdYears})"
mdoRaw.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoRaw.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoRaw.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoRaw.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the metadata object to the raw feature dataset

In [ ]:
# Apply the metadata object to the raw data feature dataset
mdRaw = md.Metadata(gdbRawData)
if not mdRaw.isReadOnly:
    mdRaw.copy(mdoRaw)
    mdRaw.save()
    print(f"Metadata updated for the raw data feature dataset.")

Metadata updated for the raw data feature dataset.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.5. Supporting Data Feature Dataset Metadata</h2>

Create a new metadata object for the supporting feature dataset

In [109]:
mdoSupporting = md.Metadata()
mdoSupporting.title = "OCSWITRS Traffic Collisions Supporting Data Dataset"
mdoSupporting.tags = "Orange County, California, OCSWITRS, Traffic, Traffic Conditions, Crashes, Collisions, Parties, Victims, Injuries, Fatalities, Hot Spots, Road Safety, Accidents, SWITRS, Transportation"
mdoSupporting.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdYears})"
mdoSupporting.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoSupporting.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoSupporting.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoSupporting.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the metadata object to the supporting feature dataset

In [111]:
# Apply the metadata object to the supporting data feature dataset
mdSupporting = md.Metadata(gdbSupportingData)
if not mdSupporting.isReadOnly:
    mdSupporting.copy(mdoSupporting)
    mdSupporting.save()
    print(f"Metadata updated for the supporting data feature dataset.")

Metadata updated for the supporting data feature dataset.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.6. Collisions Metadata</h2>

Create a new metadata object for the collisions feature class

In [112]:
# Define key metadata attributes for the Collisions feature class
mdoCollisions = md.Metadata()
mdoCollisions.title = "OCSWITRS Combined Collisions Points"
mdoCollisions.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoCollisions.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdYears})"
mdoCollisions.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoCollisions.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCollisions.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCollisions.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the collisions metadata object to the collisions feature class

In [113]:
# Apply the metadata object to the collisions feature class
mdCollisions = md.Metadata(collisions)
if not mdCollisions.isReadOnly:
    mdCollisions.copy(mdoCollisions)
    mdCollisions.save()
    print(f"Metadata updated for {collisionsAlias} feature class.")

Metadata updated for OCSWITRS Collisions feature class.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.2. Crashes Metadata</h2>

Create a new metadata object for the crashes feature class

In [114]:
# Define key metadata attributes for the Crashes feature class
mdoCrashes = md.Metadata()
mdoCrashes.title = "OCSWITRS Crashes Points"
mdoCrashes.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoCrashes.summary = f"Statewide Integrated Traffic Records System (SWITRS) Crash Data for Orange County, California ({mdYears})"
mdoCrashes.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on crashes</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoCrashes.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCrashes.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCrashes.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the crashes metadata object to the crashes feature class

In [115]:
# Apply the metadata object to the crashes feature class
mdCrashes = md.Metadata(crashes)
if not mdCrashes.isReadOnly:
    mdCrashes.copy(mdoCrashes)
    mdCrashes.save()
    print(f"Metadata updated for {crashesAlias} feature class.")

Metadata updated for OCSWITRS Crashes feature class.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.3. Parties Metadata</h2>

Create a new metadata object for the parties feature class

In [116]:
# Define key metadata attributes for the Parties feature class
mdoParties = md.Metadata()
mdoParties.title = "OCSWITRS Parties Points"
mdoParties.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Parties, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoParties.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Parties Data for Orange County, California ({mdYears})"
mdoParties.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on parties involved in crash incidents</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoParties.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoParties.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoParties.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/1e07bb1002f9457fa6fd3540fdb08e29/data"

Assign the parties metadata object to the parties feature class

In [117]:
# Apply the metadata object to the parties feature class
mdParties = md.Metadata(parties)
if not mdParties.isReadOnly:
    mdParties.copy(mdoParties)
    mdParties.save()
    print(f"Metadata updated for {partiesAlias} feature class.")

Metadata updated for OCSWITRS Parties feature class.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.4. Victims Metadata</h2>

Create a new metadata object for the victims feature class

In [118]:
# Define key metadata attributes for the Victims feature class
mdoVictims = md.Metadata()
mdoVictims.title = "OCSWITRS Victims Points"
mdoVictims.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Victims, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoVictims.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Victims Data for Orange County, California ({mdYears})"
mdoVictims.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on victims/persons involved in crash incidents</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoVictims.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoVictims.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoVictims.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/78682395df4744009c58625f1db0c25b/data"

Assign the victims metadata object to the victims feature class

In [119]:
# Apply the metadata object to the victims feature class
mdVictims = md.Metadata(victims)
if not mdVictims.isReadOnly:
    mdVictims.copy(mdoVictims)
    mdVictims.save()
    print(f"Metadata updated for {victimsAlias} feature class.")

Metadata updated for OCSWITRS Victims feature class.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.5. Roads Metadata</h2>

Create a new metadata object for the roads feature class

In [129]:
# Define key metadata attributes for the Roads feature class
mdoRoads = md.Metadata()
mdoRoads.title = "OCSWITRS Roads Network"
mdoRoads.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoRoads.summary = "All roads for Orange County, California (Primary roads and highways, secondary roads, and local roads)"
mdoRoads.description = """<div style="text-align:Left;"><div><div><p><span>The Orange County Roads Network is a comprehensive representation of all roads in the area, including primary roads and highways, secondary roads, and local roads. The data are sourced from the Orange County Department of Public Works and are updated regularly to reflect the most current road network configuration.</span></p></div></div></div>"""
mdoRoads.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoRoads.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoRoads.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"

Assign the roads metadata object to the roads feature class

In [130]:
# Apply the metadata object to the roads feature class
mdRoads = md.Metadata(roads)
if not mdRoads.isReadOnly:
    mdRoads.copy(mdoRoads)
    mdRoads.save()
    print(f"Metadata updated for {roadsAlias} feature class.")

Metadata updated for OCSWITRS Roads feature class.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.6. Census Blocks Metadata</h2>

Create a new metadata object for the census blocks feature class

In [131]:
# Define key metadata attributes for the US Census 2020 Blocks feature class
mdoBlocks = md.Metadata()
mdoBlocks.title = "OCSWITRS US Census 2020 Blocks"
mdoBlocks.tags = "Orange County, California, US Census 2020, Blocks, Census, Demographics, Population"
mdoBlocks.summary = "US Census 2020 Blocks for Orange County, California"
mdoBlocks.description = """<div style="text-align:Left;"><div><div><p><span>The US Census 2020 Blocks feature class provides a comprehensive representation of the 2020 Census Blocks for Orange County, California. The data are sourced from the US Census Bureau and are updated regularly to reflect the most current demographic and population data.</span></p></div></div></div>"""
mdoBlocks.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoBlocks.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoBlocks.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/e2c4cd39783a4d1bb0925ead15a23cdc/data"

Assign the census blocks metadata object to the census blocks feature class

In [132]:
# Apply the metadata object to the US Census 2020 Blocks feature class
mdBlocks = md.Metadata(blocks)
if not mdBlocks.isReadOnly:
    mdBlocks.copy(mdoBlocks)
    mdBlocks.save()
    print(f"Metadata updated for {blocksAlias} feature class.")

Metadata updated for OCSWITRS Census Blocks feature class.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.7. Cities Metadata</h2>

Create a new metadata object for the cities feature class

In [138]:
# Define key metadata attributes for the Cities feature class
mdoCities = md.Metadata()
mdoCities.title = "OCSWITRS Cities Boundaries"
mdoCities.tags = "Orange County, California, Cities, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoCities.summary = "Orange County City and Unincorporated Areas Land Boundaries, enriched with geodemographic characteristics"
mdoCities.description = """<div style="text-align:Left;"><div><div><p><span>The Orange County City and Unincorporated Areas Land Boundaries are enriched with a comprehensive set of geodemographic characteristics from OC ACS 2021 data. These characteristics span across demographic, housing, economic, and social aspects, providing a holistic view of the area. </span></p><p><span>The geodemographic data originate from the US Census American Community Survey (ACS) 2021, a 5-year estimate of the key Characteristics of Cities' geographic level in Orange County, California. The data contains:</span></p><ul><li><span>Total population and housing counts for each area;</span></li><li><span>Population and housing density measurements (per square mile);</span></li><li><span>Race counts for Asian, Black or African American, Hispanic and White groups;</span></li><li><span>Aggregate values for the number of vehicles commuting and travel time to work;</span></li></ul></div></div></div>"""
mdoCities.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCities.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCities.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/ffe4a73307a245eda7dc7eaffe1db6d2/data"

Assign the cities metadata object to the cities feature class

In [139]:
# Apply the metadata object to the Cities feature class
mdCities = md.Metadata(cities)
if not mdCities.isReadOnly:
    mdCities.copy(mdoCities)
    mdCities.save()
    print(f"Metadata updated for {citiesAlias} feature class.")

Metadata updated for OCSWITRS Cities feature class.


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.8. Boundaries Metadata</h2>

Create a new metadata object for the boundaries feature class

In [135]:
# Define key metadata attributes for the Boundaries feature class
mdoBoundaries = md.Metadata()
mdoBoundaries.title = "OC Land Boundaries"
mdoBoundaries.tags = "Orange County, California, Boundaries, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoBoundaries.summary = "Land boundaries for Orange County, cities, and unincorporated areas"
mdoBoundaries.description = """<div style="text-align:Left;"><div><div><p><span>Land boundaries for Orange County, cities, and unincorporated areas (based on the five supervisorial districts). Contains additional geodemographic data on population and housing from the US Census 2021 American Community Survey (ACS).</span></p></div></div></div>"""
mdoBoundaries.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoBoundaries.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoBoundaries.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/4041c4b1f4234218a4ce654e5d22f176/data"

Assign the boundaries metadata object to the boundaries feature class

In [136]:
# Apply the metadata object to the Boundaries feature class
mdBoundaries = md.Metadata(boundaries)
if not mdBoundaries.isReadOnly:
    mdBoundaries.copy(mdoBoundaries)
    mdBoundaries.save()
    print(f"Metadata updated for {boundariesAlias} feature class.")

Metadata updated for OCSWITRS Boundaries feature class.


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [140]:
# Save the project
aprx.save()

<div style = "background-color:indigo"><center>
<h1 style="font-weight:bold; color:goldenrod; border-top: 2px solid goldenrod; border-bottom: 2px solid goldenrod; padding-top: 5px; padding-bottom: 10px">End of Script</h1>
</center></div>